In [11]:
cd Desktop/Summer\ research/fem/

[Errno 2] No such file or directory: 'Desktop/Summer research/fem/'
/Users/wuyifan/Desktop/Summer research/fem


In [2]:
import numpy as np
import meshplot as mp

import wildmeshing as wm
import polyfempy as pf

In [3]:
V = np.load("data/V.npy")
L = np.load("data/L.npy")

"""
p=mp.plot(V, np.zeros((0,3)), return_plot=True)
p.add_edges(V, L, shading={});
p.add_points(V, shading={"point_color": "red", "point_size": 10})
p.save("Load.html")
"""

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(194.30000…

2

In [10]:
vs, tris, nodes, F_nodes = wm.triangulate_svg("data/planehole.svg", cut_outside=True)

p = mp.plot(vs, tris,shading={"wireframe": True}, return_plot=True)
p.add_points(V, shading={"point_color": "red", "point_size": 5});
p.add_points(nodes, shading={"point_color": "green", "point_size": 10})
p.save("mesh.html")


1/1 n sub paths: 5


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(194.29999…

Plot saved to file mesh.html.


In [16]:
solver = pf.Solver()

# vertices: array, connectivity: array, nodes_pos: array, nodes_indices: List[List[int]]
# Loads an high order mesh from vertices, connectivity, nodes, and node indices mapping element to nodes
solver.set_high_order_mesh(vs, tris, nodes, F_nodes)

# sp: points
# sl: lines
# ->tuple, exports get the boundary sideset, edges in 2d or trangles in 3d
sp, sl, s = solver.get_boundary_sidesets()
p=mp.plot(sp, np.zeros((0,3)), return_plot=True)
p.add_edges(sp, sl);

e1 = sl[s[:,0] == 1, :]
e2 = sl[s[:,0] == 2, :]
e3 = sl[s[:,0] == 3, :]
e4 = sl[s[:,0] == 4, :]
e5 = sl[s[:,0] == 7, :]

p.add_points((sp[e1[:, 0], :]+sp[e1[:, 1], :])/2, shading={"point_color": "red", "point_size": 5});
p.add_points((sp[e5[:, 0], :]+sp[e5[:, 1], :])/2, shading={"point_color": "yellow", "point_size": 5});
p.add_points((sp[e2[:, 0], :]+sp[e2[:, 1], :])/2, shading={"point_color": "green", "point_size": 5});
p.add_points((sp[e3[:, 0], :]+sp[e3[:, 1], :])/2, shading={"point_color": "blue", "point_size": 5});
p.add_points((sp[e4[:, 0], :]+sp[e4[:, 1], :])/2, shading={"point_color": "black", "point_size": 5})
p.save("bound.html")

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(194.30000…

[1. 7. 1. 4. 7. 7. 7. 7. 2. 7. 2. 3. 4. 3. 1. 1. 1. 1. 4. 4. 4. 4. 4. 4.
 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 3. 3. 3. 3. 3. 3. 3. 3.]
[[  0   1]
 [  2   3]
 [  4   5]
 [  6   7]
 [  8   9]
 [ 10  11]
 [ 12  13]
 [ 14  15]
 [ 16  17]
 [ 18  19]
 [ 20  21]
 [ 22  23]
 [ 24  25]
 [ 26  27]
 [ 28  29]
 [ 30  31]
 [ 32  33]
 [ 34  35]
 [ 36  37]
 [ 38  39]
 [ 40  41]
 [ 42  43]
 [ 44  45]
 [ 46  47]
 [ 48  49]
 [ 50  51]
 [ 52  53]
 [ 54  55]
 [ 56  57]
 [ 58  59]
 [ 60  61]
 [ 62  63]
 [ 64  65]
 [ 66  67]
 [ 68  69]
 [ 70  71]
 [ 72  73]
 [ 74  75]
 [ 76  77]
 [ 78  79]
 [ 80  81]
 [ 82  83]
 [ 84  85]
 [ 86  87]
 [ 88  89]
 [ 90  91]
 [ 92  93]
 [ 94  95]
 [ 96  97]
 [ 98  99]
 [100 101]
 [102 103]
 [104 105]
 [106 107]
 [108 109]
 [110 111]
 [112 113]]
[[ 0  1]
 [ 4  5]
 [28 29]
 [30 31]
 [32 33]
 [34 35]]
Plot saved to file bound.html.


In [76]:
settings = pf.Settings()
problem = pf.Problem()

# Sets the PDE to solve, use any of the polyfempy.PDEs
settings.set_pde(pf.PDEs.NonLinearElasticity)

settings.discr_order = 2

# set the material parameters
# sets the Young’s modulus E to 210000
settings.set_material_params("E", 210000)
settings.set_material_params("nu", 0.3)

# x coorinate is fixed, y is allowed to move (Neumann)
problem.set_x_symmetric(1)
problem.set_y_symmetric(2)

# set the *force* value for the sideset *id*
problem.set_force(3, [100, 0])

settings.set_problem(problem)

solver.settings(settings)
solver.solve()

# returns the solution on a densly sampled mesh
sol_pts, sol_tri, disp = solver.get_sampled_solution()

# returns the von mises stresses and stress tensor averaged around a vertex on a densly sampled mesh
misises, _ = solver.get_sampled_mises_avg()

a = mp.plot(sol_pts + disp, sol_tri, misises, return_plot=True)
a.save("solve.html")

[2020-05-18 15:11:15.251] [polyfem] [info] simplex_count: 	363
[2020-05-18 15:11:15.251] [polyfem] [info] regular_count: 	0
[2020-05-18 15:11:15.251] [polyfem] [info] regular_boundary_count: 	0
[2020-05-18 15:11:15.251] [polyfem] [info] simple_singular_count: 	0
[2020-05-18 15:11:15.251] [polyfem] [info] multi_singular_count: 	0
[2020-05-18 15:11:15.251] [polyfem] [info] boundary_count: 	0
[2020-05-18 15:11:15.251] [polyfem] [info] multi_singular_boundary_count: 	0
[2020-05-18 15:11:15.251] [polyfem] [info] non_regular_count: 	0
[2020-05-18 15:11:15.251] [polyfem] [info] non_regular_boundary_count: 	0
[2020-05-18 15:11:15.252] [polyfem] [info] undefined_count: 	0
[2020-05-18 15:11:15.257] [polyfem] [info] total count:	 363
[2020-05-18 15:11:15.257] [polyfem] [info] Building not isoparametric basis...
[2020-05-18 15:11:15.284] [polyfem] [info] Computing polygonal basis...
[2020-05-18 15:11:15.285] [polyfem] [info]  took 0.000390718s
[2020-05-18 15:11:15.285] [polyfem] [info] hmin: 3.718

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(194.32994…

Plot saved to file solve.html.
